In [2]:
import os 
import pandas as pd

In [3]:
# Specify the directory
directory = 'training_set/agn_spectra'

# Get a list of all files in the directory
id_train_agn = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# star directory
directory = 'training_set/star_spectra'

# Get a list of all files in the directory
id_train_star = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# binary directory
directory = 'training_set/bin_spectra'

# Get a list of all files in the directory
id_train_bin = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# galaxy directory
directory = 'training_set/gal_spectra'

# Get a list of all files in the directory
id_train_gal = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# print the number of files in each category
print('Number of AGN spectra:', len(id_train_agn))
print('Number of STAR spectra:', len(id_train_star))
print('Number of BIN spectra:', len(id_train_bin))
print('Number of GAL spectra:', len(id_train_gal))




Number of AGN spectra: 35936
Number of STAR spectra: 86037
Number of BIN spectra: 40676
Number of GAL spectra: 1699


In [4]:
# Specify the directory
directory = 'validation_set/agn_spectra'

# Get a list of all files in the directory
id_val_agn = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# star directory
directory = 'validation_set/star_spectra'

# Get a list of all files in the directory
id_val_star = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# binary directory
directory = 'validation_set/bin_spectra'

# Get a list of all files in the directory
id_val_bin = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# galaxy directory
directory = 'validation_set/gal_spectra'

# Get a list of all files in the directory
id_val_gal = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# print the number of files in each category
print('Number of AGN spectra:', len(id_val_agn))
print('Number of STAR spectra:', len(id_val_star))
print('Number of BIN spectra:', len(id_val_bin))
print('Number of GAL spectra:', len(id_val_gal))


Number of AGN spectra: 400
Number of STAR spectra: 400
Number of BIN spectra: 400
Number of GAL spectra: 400


# Getting the ra, dec from obsid list

In [5]:
lamost_catalog = pd.read_csv("dr9_v2.0_LRS_catalogue.csv")  # Assuming CSV format for LAMOST catalog



### Starting of with val gal

In [11]:
obsid_list = id_val_gal
obsid_list = [int(obsid) for obsid in obsid_list]

# Filter the DataFrame
filtered_df = lamost_catalog[lamost_catalog['obsid'].isin(obsid_list)]

# Get the 'ra' and 'dec' values
ra_dec_values = filtered_df[['ra', 'dec']]
print(ra_dec_values)

                  ra        dec
1838      333.865510   0.776690
113126    138.383021  31.957768
237337     42.455140  -0.324820
285166    175.558822  28.841412
288652     20.021740  -0.404700
...              ...        ...
10491764  246.803116  29.998576
10567385  117.924140  53.057106
10595660  227.251731  31.032884
10669548  115.395570  23.501689
10681656  189.348344  55.694221

[400 rows x 2 columns]


## Cross matching with gaia back

In [ ]:
# should i be worried with nans?